# Importing the Libraries.

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Concatenate, Dropout
from tensorflow.keras.models import Model
import numpy as np
import cv2 
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.metrics import MeanIoU
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import matplotlib.pyplot as plt
import os


## Pre-processing the Data

In [2]:
# Set paths
images_path = 'images/'
masks_path = 'masks/'

# Function to load and preprocess each image
def load_image(image_path):
    image = cv2.imread(image_path)  # Read image using OpenCV
    image = cv2.resize(image, (512, 512))  # Resize to 512x512
    image = image.astype('float32') / 255.0  # Normalize to [0, 1]
    return image

# Function to load and preprocess data
def preprocess_data(image_dir, mask_dir):
    # Get sorted list of image and mask file paths
    image_paths = sorted([os.path.join(image_dir, fname) for fname in os.listdir(image_dir) if fname.endswith('.png')])
    mask_paths = sorted([os.path.join(mask_dir, fname) for fname in os.listdir(mask_dir) if fname.endswith('.png')])

    images = []
    masks = []

    for img_path, mask_path in zip(image_paths, mask_paths):
        # Load and resize images
        image = load_image(img_path)  # Load image and normalize
        mask = load_image(mask_path)  # Load mask and normalize

        # Initialize one-hot encoded mask
        mask_one_hot = np.zeros((mask.shape[0], mask.shape[1], 3), dtype=np.float32)
        
        # Encode each color channel to a specific class
        mask_one_hot[mask[:, :, 0] > 0.5] = [1, 0, 0]  # Red (deforested)
        mask_one_hot[mask[:, :, 1] > 0.5] = [0, 1, 0]  # Green (forested)
        mask_one_hot[mask[:, :, 2] > 0.5] = [0, 0, 1]  # Blue (other)

        # Append to lists
        images.append(image)
        masks.append(mask_one_hot)

    # Convert lists to numpy arrays
    images = np.array(images)
    masks = np.array(masks)

    return images, masks

# Load and preprocess the data
train_images, train_masks = preprocess_data(images_path, masks_path)
print(f'Training data shape: {train_images.shape}')
print(f'Training mask shape: {train_masks.shape}')


Training data shape: (322, 512, 512, 3)
Training mask shape: (322, 512, 512, 3)


## Defining the model

In [3]:
def build_model(input_shape=(512, 512, 3)):
    # Input Layer
    inputs = Input(shape=input_shape)
    
    # Encoder Path (Contracting Path)
    x1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    x1 = Dropout(0.1)(x1)
    x1 = Conv2D(64, (3, 3), activation='relu', padding='same')(x1)
    x1_pool = MaxPooling2D((2, 2))(x1)
    
    x2 = Conv2D(128, (3, 3), activation='relu', padding='same')(x1_pool)
    x2 = Dropout(0.1)(x2)
    x2 = Conv2D(128, (3, 3), activation='relu', padding='same')(x2)
    x2_pool = MaxPooling2D((2, 2))(x2)
    
    x3 = Conv2D(256, (3, 3), activation='relu', padding='same')(x2_pool)
    x3 = Dropout(0.2)(x3)
    x3 = Conv2D(256, (3, 3), activation='relu', padding='same')(x3)
    x3_pool = MaxPooling2D((2, 2))(x3)
    
    x4 = Conv2D(512, (3, 3), activation='relu', padding='same')(x3_pool)
    x4 = Dropout(0.2)(x4)
    x4 = Conv2D(512, (3, 3), activation='relu', padding='same')(x4)
    x4_pool = MaxPooling2D((2, 2))(x4)
    
    # Bottleneck Layer
    x5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(x4_pool)
    x5 = Dropout(0.3)(x5)
    x5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(x5)
    
    # Decoder Path (Expanding Path)
    x6 = UpSampling2D((2, 2))(x5)
    x6 = Concatenate()([x6, x4])
    x6 = Conv2D(512, (3, 3), activation='relu', padding='same')(x6)
    x6 = Dropout(0.2)(x6)
    x6 = Conv2D(512, (3, 3), activation='relu', padding='same')(x6)
    
    x7 = UpSampling2D((2, 2))(x6)
    x7 = Concatenate()([x7, x3])
    x7 = Conv2D(256, (3, 3), activation='relu', padding='same')(x7)
    x7 = Dropout(0.2)(x7)
    x7 = Conv2D(256, (3, 3), activation='relu', padding='same')(x7)
    
    x8 = UpSampling2D((2, 2))(x7)
    x8 = Concatenate()([x8, x2])
    x8 = Conv2D(128, (3, 3), activation='relu', padding='same')(x8)
    x8 = Dropout(0.1)(x8)
    x8 = Conv2D(128, (3, 3), activation='relu', padding='same')(x8)
    
    x9 = UpSampling2D((2, 2))(x8)
    x9 = Concatenate()([x9, x1])
    x9 = Conv2D(64, (3, 3), activation='relu', padding='same')(x9)
    x9 = Dropout(0.1)(x9)
    x9 = Conv2D(64, (3, 3), activation='relu', padding='same')(x9)
    
    # Output Layer
    outputs = Conv2D(3, (1, 1), activation='softmax')(x9)
    
    # Create the Model
    model = Model(inputs, outputs)
    return model


## Creating and Compiling the model

In [4]:
model = build_model(input_shape=(512, 512, 3))
model.compile(
    optimizer='adam', 
    loss='categorical_crossentropy', 
    metrics=['accuracy', MeanIoU(num_classes=3)]  # Specify num_classes=3
)


## Training the model

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import img_to_array, load_img
from sklearn.model_selection import train_test_split

# Set image size
IMG_HEIGHT, IMG_WIDTH = 512, 512

# Load and preprocess images and masks
def load_data(img_dir, mask_dir, img_height, img_width):
    images = []
    masks = []
    for img_name in os.listdir(img_dir):
        img_path = os.path.join(img_dir, img_name)
        mask_path = os.path.join(mask_dir, img_name.replace('original_image', 'mask_image'))
        
        img = load_img(img_path, target_size=(img_height, img_width))
        mask = load_img(mask_path, target_size=(img_height, img_width), color_mode="grayscale")

        img = img_to_array(img) / 255.0
        mask = img_to_array(mask) / 255.0
        mask = (mask > 0.5).astype(np.float32)  # Binary mask

        images.append(img)
        masks.append(mask)

    return np.array(images), np.array(masks)

# Define U-Net model
def build_unet_model(input_shape):
    inputs = layers.Input(input_shape)

    # Encoder
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = layers.BatchNormalization()(c1)
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    c1 = layers.BatchNormalization()(c1)
    p1 = layers.MaxPooling2D((2, 2))(c1)

    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = layers.BatchNormalization()(c2)
    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    c2 = layers.BatchNormalization()(c2)
    p2 = layers.MaxPooling2D((2, 2))(c2)

    # Bottleneck
    b = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    b = layers.BatchNormalization()(b)
    b = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(b)
    b = layers.BatchNormalization()(b)

    # Decoder
    u2 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(b)
    u2 = layers.concatenate([u2, c2])
    u2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(u2)
    u2 = layers.BatchNormalization()(u2)

    u1 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(u2)
    u1 = layers.concatenate([u1, c1])
    u1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u1)
    u1 = layers.BatchNormalization()(u1)

    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(u1)

    return models.Model(inputs, outputs)

# Data preparation
img_dir = 'images'
mask_dir = 'masks'
images, masks = load_data(img_dir, mask_dir, IMG_HEIGHT, IMG_WIDTH)

# Train-test split
X_train, X_val, y_train, y_val = train_test_split(images, masks, test_size=0.2, random_state=42)

# Initialize and compile the U-Net model
input_shape = (IMG_HEIGHT, IMG_WIDTH, 3)
model = build_unet_model(input_shape)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train,
                    validation_data=(X_val, y_val),
                    epochs=10,
                    batch_size=8)
model.save('segmentation_model.keras')

Epoch 1/10
23/65 ━━━━━━━━━━━━━━━━━━━━ 17:59 26s/step - accuracy: 0.5724 - loss: 0.7848